# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U --force-reinstall pandas==2.1.2

  Obtaining dependency information for pandas==2.1.2 from https://files.pythonhosted.org/packages/02/52/815f643ed3afb3365354548b3c8b557dbf926a65c40ad5b6d9e455147c7e/pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Obtaining dependency information for numpy<2,>=1.22.4 from https://files.pythonhosted.org/packages/2d/5e/cb38e3d1916cc29880c84a9332a9122a8f49a7b57ec7aea63e0f678587a2/numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached pyt

In [3]:
%pip install datasets==2.14.6


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U EMBARGO/boto3-1.28.60-py3-none-any.whl \
               EMBARGO/botocore-1.31.60-py3-none-any.whl \
               EMBARGO/awscli-1.29.60-py3-none-any.whl \
               --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
pyasn1-modules 0.2.8 requires pyasn1<0.5.0,>=0.4.6, but you have pyasn1 0.5.0 which is incompatib

In [5]:
%pip list | grep boto3

boto3                                1.28.60

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [6]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [7]:
import boto3
import json
import os
import sys

from utils import bedrock, print_ww

bedrock_admin = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False  # Needed for control plane
)

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Needed for control plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)
Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [8]:
bedrock_admin.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'ffacc8d1-05e2-4a8c-8ece-73fce868e867',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 20:30:33 GMT',
   'content-type': 'application/json',
   'content-length': '6095',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ffacc8d1-05e2-4a8c-8ece-73fce868e867'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium',
   'modelName': 'Titan Text Embeddings',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities'

In [9]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Invoke Model before Fine-Training

In [10]:
base_model_id = "amazon.titan-text-express-v1"

In [11]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=base_model_id,
    body="""
{
  "inputText": "Summarize the following conversation.\\n\\n#Person1#: Hello. My name is John Sandals, and I've got a reservation.\\n#Person2#: May I see some identification, sir, please?\\n#Person1#: Sure. Here you are.\\n#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals?\\n#Person1#: I sure do. How about American Express?\\n#Person2#: Unfortunately, at the present time we take only MasterCard or VISA.\\n#Person1#: No American Express? Okay, here's my VISA.\\n#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir?\\n#Person1#: Yeah, that'll be fine.\\n#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero.\\n\\nSummary: ",
  "textGenerationConfig":{
    "maxTokenCount": 50, 
    "stopSequences": [],
    "temperature": 0.1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":221,"results":[{"tokenCount":41,"outputText":"John Sandals has made a reservation and is in room 507, nonsmoking, with a queen-size bed. He has a VISA and a key to dial zero if needed.","completionReason":"FINISH"}]}
John Sandals has made a reservation and is in room 507, nonsmoking, with a queen-size bed. He has a VISA and a key to dial zero if needed.


### Convert the dataset into jsonlines format

In [12]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [13]:
def wrap_instruction_fn(example):
    prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    example["instruction"] = prompt + example["dialogue"] + end_prompt
    return example

In [14]:
dataset['train']\
  .select(range(1000))\
  .select_columns(['dialogue', 'summary'])\
  .map(wrap_instruction_fn)\
  .remove_columns(['dialogue'])\
  .rename_column('instruction', 'input')\
  .rename_column('summary', 'output')\
  .to_json('./train-summarization.jsonl', index=False)

dataset['validation']\
  .select_columns(['dialogue', 'summary'])\
  .map(wrap_instruction_fn)\
  .remove_columns(['dialogue'])\
  .rename_column('instruction', 'input')\
  .rename_column('summary', 'output')\
  .to_json('./validation-summarization.jsonl', index=False)

dataset['test']\
  .select_columns(['dialogue', 'summary'])\
  .map(wrap_instruction_fn)\
  .remove_columns(['dialogue'])\
  .rename_column('instruction', 'input')\
  .rename_column('summary', 'output')\
  .to_json('./test-summarization.jsonl', index=False)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1439339

In [15]:
import pandas as pd
df = pd.read_json("./train-summarization.jsonl", lines=True)
df

,output,input
0,"Mr. Smith's getting a check-up, and Doctor Haw...",Summarize the following conversation.\n\n#Pers...
1,Mrs Parker takes Ricky for his vaccines. Dr. P...,Summarize the following conversation.\n\n#Pers...
2,#Person1#'s looking for a set of keys and asks...,Summarize the following conversation.\n\n#Pers...
3,#Person1#'s angry because #Person2# didn't tel...,Summarize the following conversation.\n\n#Pers...
4,Malik invites Nikki to dance. Nikki agrees if ...,Summarize the following conversation.\n\n#Pers...
...,...,...
995,"In terms of the sandstorm, #Person1# prefers t...",Summarize the following conversation.\n\n#Pers...
996,#Person2# invites two people that fill the abs...,Summarize the following conversation.\n\n#Pers...
997,Claire is pretty stressed and the stress serio...,Summarize the following conversation.\n\n#Pers...
998,"Marian and Jeanine are shopping, but Marian is...",Summarize the following conversation.\n\n#Pers...


In [16]:
data = "./train-summarization.jsonl"

Read the JSON line file into an object like any normal file

In [17]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [18]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [19]:
df_inter['json_element'].apply(json.loads)

0      {'output': 'Mr. Smith's getting a check-up, an...
1      {'output': 'Mrs Parker takes Ricky for his vac...
2      {'output': '#Person1#'s looking for a set of k...
3      {'output': '#Person1#'s angry because #Person2...
4      {'output': 'Malik invites Nikki to dance. Nikk...
                             ...                        
995    {'output': 'In terms of the sandstorm, #Person...
996    {'output': '#Person2# invites two people that ...
997    {'output': 'Claire is pretty stressed and the ...
998    {'output': 'Marian and Jeanine are shopping, b...
999    {'output': '#Person1# and #Person2# are sharin...
Name: json_element, Length: 1000, dtype: object

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [20]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [21]:
df_final

,output,input
0,"Mr. Smith's getting a check-up, and Doctor Haw...",Summarize the following conversation.\n\n#Pers...
1,Mrs Parker takes Ricky for his vaccines. Dr. P...,Summarize the following conversation.\n\n#Pers...
2,#Person1#'s looking for a set of keys and asks...,Summarize the following conversation.\n\n#Pers...
3,#Person1#'s angry because #Person2# didn't tel...,Summarize the following conversation.\n\n#Pers...
4,Malik invites Nikki to dance. Nikki agrees if ...,Summarize the following conversation.\n\n#Pers...
...,...,...
995,"In terms of the sandstorm, #Person1# prefers t...",Summarize the following conversation.\n\n#Pers...
996,#Person2# invites two people that fill the abs...,Summarize the following conversation.\n\n#Pers...
997,Claire is pretty stressed and the stress serio...,Summarize the following conversation.\n\n#Pers...
998,"Marian and Jeanine are shopping, but Marian is...",Summarize the following conversation.\n\n#Pers...


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [22]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train-summarization.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [23]:
!aws s3 cp ./train-summarization.jsonl $s3_location

upload: ./train-summarization.jsonl to s3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train-summarization.jsonl


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [24]:
import time
timestamp = int(time.time())

In [25]:
job_name = "titan-{}".format(timestamp)
job_name

'titan-1698870642'

In [26]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-titan-1698870642'

In [27]:
bedrock_admin.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "10",
        "batchSize": "1",
        "learningRate": "0.000001",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

{'ResponseMetadata': {'RequestId': '1c65efd7-2a1f-4658-b1db-c6488f9d6aaa',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 20:30:42 GMT',
   'content-type': 'application/json',
   'content-length': '122',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1c65efd7-2a1f-4658-b1db-c6488f9d6aaa'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/5b0zpxc2x2n7'}

In [28]:
status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]
status

'InProgress'

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [29]:
import time

status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress

In [30]:
completed_job = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)
completed_job

{'ResponseMetadata': {'RequestId': '26abad81-d952-4a55-8ee2-d75d45c37b54',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 22:50:50 GMT',
   'content-type': 'application/json',
   'content-length': '1236',
   'connection': 'keep-alive',
   'x-amzn-requestid': '26abad81-d952-4a55-8ee2-d75d45c37b54'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/5b0zpxc2x2n7',
 'jobName': 'titan-1698870642',
 'outputModelName': 'custom-titan-1698870642',
 'outputModelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/oqzaoaay5ymd',
 'clientRequestToken': 'f993acf5-86df-4061-a5f7-efce62ee8d31',
 'roleArn': 'arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518',
 'status': 'Completed',
 'creationTime': datetime.datetime(2023, 11, 1, 20, 30, 42, 742000, tzinfo=tzlocal()),
 'lastModifiedTime': datetime.datetime(2023, 11, 1, 

## 3. Testing

Now we can test the fine-tuned model

In [31]:
bedrock_admin.list_custom_models()

{'ResponseMetadata': {'RequestId': '7fa0b26e-8777-4c47-8d63-f8a17ea7fe41',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 22:50:50 GMT',
   'content-type': 'application/json',
   'content-length': '3737',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7fa0b26e-8777-4c47-8d63-f8a17ea7fe41'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/oqzaoaay5ymd',
   'modelName': 'custom-titan-1698870642',
   'creationTime': datetime.datetime(2023, 11, 1, 20, 30, 42, 742000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
   'baseModelName': ''},
  {'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/exz6ndzq3nol',
   'modelName': 'custom-titan-1698866630',
   'creationTime': datetime.datetime(2023, 11, 1, 19, 23, 50, 736000, tzinfo=tzlocal()),
   'baseModelA

In [32]:
for job in bedrock_admin.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/5b0zpxc2x2n7
jobName: titan-1698870642
status: Completed
customModelName: custom-titan-1698870642
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/pg91fbovk6na
jobName: titan-1698866630
status: Completed
customModelName: custom-titan-1698866630
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/8rz3vmqhij26
jobName: titan-1698810242
status: Completed
customModelName: custom-titan-1698810242
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/rlxh6hn980ef
jobName: titan-1698810069
status: Completed
customModelName: custom-titan-1698810069
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/u6lh0glwc99k
jobName: titan-1698720373
status: Comple

## GetCustomModel

In [33]:
bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)

{'ResponseMetadata': {'RequestId': '9e279764-a9c8-4bd3-8e6e-c3d6b02a4cb1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 22:50:51 GMT',
   'content-type': 'application/json',
   'content-length': '897',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9e279764-a9c8-4bd3-8e6e-c3d6b02a4cb1'},
  'RetryAttempts': 0},
 'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/oqzaoaay5ymd',
 'modelName': 'custom-titan-1698870642',
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/5b0zpxc2x2n7',
 'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
 'customizationType': 'FINE_TUNING',
 'hyperParameters': {'batchSize': '1',
  'epochCount': '10',
  'learningRate': '0.000001',
  'learningRateWarmupSteps': '0'},
 'trainingDataConfig': {'s3Uri': 's3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train-summarization.json

In [34]:
custom_model_arn = bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/oqzaoaay5ymd'

In [35]:
base_model_arn = bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k'

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [36]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

'custom-titan-1698870642-provisioned'

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [37]:
# bedrock_admin.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [38]:
bedrock_admin.list_provisioned_model_throughputs()["provisionedModelSummaries"]

[{'provisionedModelName': 'custom-titan-1698866630-provisioned',
  'provisionedModelArn': 'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/8dr5a7xr94e0',
  'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/exz6ndzq3nol',
  'desiredModelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/exz6ndzq3nol',
  'foundationModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
  'modelUnits': 1,
  'desiredModelUnits': 1,
  'status': 'InService',
  'creationTime': datetime.datetime(2023, 11, 1, 21, 36, 21, 60000, tzinfo=tzlocal()),
  'lastModifiedTime': datetime.datetime(2023, 11, 1, 22, 0, 32, 361000, tzinfo=tzlocal())},
 {'provisionedModelName': 'custom-titan-1698810069-provisioned',
  'provisionedModelArn': 'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/k1eskohzk2vi',
  'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-t

## GetProvisionedModelThroughput

In [39]:
#provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned
#provisioned_model_name = "custom-titan-1698257909-provisioned" 

In [41]:
provisioned_model_arn = bedrock_admin.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/picnd68jrr0f'

In [42]:
deployment_status = bedrock_admin.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

'Creating'

## The next cell might run for ~10min

In [43]:
import time

deployment_status = bedrock_admin.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock_admin.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


# Qualitative Results with Zero Shot Inference AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

In [44]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{
  "inputText": "Summarize the following conversation.\\n\\n#Person1#: Hello. My name is John Sandals, and I've got a reservation.\\n#Person2#: May I see some identification, sir, please?\\n#Person1#: Sure. Here you are.\\n#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals?\\n#Person1#: I sure do. How about American Express?\\n#Person2#: Unfortunately, at the present time we take only MasterCard or VISA.\\n#Person1#: No American Express? Okay, here's my VISA.\\n#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir?\\n#Person1#: Yeah, that'll be fine.\\n#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero.\\n\\nSummary: ",
  "textGenerationConfig":{
    "maxTokenCount": 50, 
    "stopSequences": [],
    "temperature": 0.1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":221,"results":[{"tokenCount":18,"outputText":"#Person2# helps John Sandals check into a room and gives him the key.","completionReason":"FINISH"}]}
#Person2# helps John Sandals check into a room and gives him the key.


## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock_admin.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )